In [11]:
from src import Tensor, Sum
import numpy as np
from sklearn.datasets import load_digits

digits = load_digits()
ixx = np.arange(len(digits['data']))
np.random.shuffle(ixx)

for ix in ixx:
    x = digits['data'][ix]
    y = digits['target'][ix]
    break

In [2]:
class Linear():
    def __init__(self, input_dim, output_dim, bias=True):
        self.bias = bias
        self.m = Tensor(np.random.rand(output_dim, input_dim + int(bias)))

    def __call__(self, x):
        if self.bias:
            x_with_extra_1 = np.ones(x.shape)


class Model():
    def __init__(self):
        self.m1 = Tensor(np.random.rand(32,64))
        self.m2 = Tensor(np.random.rand(10,32))

    def forward(self, x):
        out = self.m1 @ x
        out = out * (out > Tensor(0))
        out = self.m2 @ out

        z = out - Tensor(out.value.max())
        exps = Tensor(np.e, requires_grad=False) ** out

        softmax = exps / Sum(exps)

        return softmax



In [44]:
model = Model()

x = (digits['data'][ix] / 16).reshape(-1, 1)
y = np.zeros((10,1))
y[digits['target'][ix],:] = 1

x = Tensor(x, requires_grad=False)
y = Tensor(y, requires_grad=False)

out = model.forward(x)

loss = Sum((out - y) ** Tensor(2, requires_grad=False))

loss.backwards(np.ones(loss.value.shape))

before = loss.value
before

c:\Users\Rens\repos\neural_networks\src.py:74: RuntimeWarning: invalid value encountered in log
  self.b.backwards(np.log(self.a.value) * self.a.value ** self.b.value * accumulated_gradient)


array([0.15399165])

In [48]:
model.m2.grad[3,0]

-0.8930304753086191

In [49]:
nudge = np.zeros(model.m2.value.shape)
delta = 0.0001
nudge[3,0] = delta

model.m2.value = model.m2.value + nudge

In [50]:
out = model.forward(x)

loss = Sum((out - y) ** Tensor(2, requires_grad=False))

after = loss.value
after

array([0.15381316])

In [51]:
(after - before) / delta

array([-1.78494995])

In [10]:
from scipy.signal import medfilt

model = Model()

lr = 0.1
losses = [] 
batch_size = 64
batch_loss = []

for epoch in range(20):
    for point, ix in enumerate(ixx):
        x = (digits['data'][ix] / 16).reshape(-1, 1)
        y = np.zeros((10,1))
        y[digits['target'][ix],:] = 1
        
        x = Tensor(x, requires_grad=False)
        y = Tensor(y, requires_grad=False)

        y_hat = model.forward(x)


        loss = Sum((y_hat - y) ** Tensor(2, requires_grad=False))
        losses.append(loss.value[0])

        loss.backwards(np.ones(loss.value.shape))

        if point % batch_size == 0:
            batch_loss.append(np.mean(losses[-64:]))
            for m in [model.m1]:
                m.value = m.value - lr * m.grad
                m.grad = np.zeros(m.value.shape)

    
plt.plot(range(len(batch_loss)), batch_loss, alpha=0.5)

c:\Users\Rens\repos\neural_networks\src.py:74: RuntimeWarning: invalid value encountered in log
  self.b.backwards(np.log(self.a.value) * self.a.value ** self.b.value * accumulated_gradient)


NameError: name 'plt' is not defined

In [176]:
y_hat.value

array([[1.40299656e-16],
       [6.78785954e-17],
       [3.57005342e-09],
       [2.12554906e-21],
       [4.83220073e-23],
       [5.10446336e-17],
       [4.70104924e-22],
       [9.99999932e-01],
       [2.41118438e-27],
       [6.47508867e-08]])

In [177]:
y.value

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [151]:
y_hat.value

array([[9.25828803e-05],
       [2.63058412e-04],
       [4.87459122e-07],
       [2.27067981e-03],
       [7.92080467e-06],
       [9.69198148e-01],
       [3.59383007e-21],
       [1.64374236e-03],
       [4.41220866e-04],
       [2.60821598e-02]])

In [153]:
y.value

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [13]:
y_hat.backwards(accum_grad)

()
[ 1.45742200e-33  2.04064664e-21 -4.41725722e-01  2.83721110e-03
  6.04229144e-18  4.65374660e-17  1.34126485e-07  1.06978308e+02
  2.57718467e+00  2.41725896e-13]


ValueError: non-broadcastable output operand with shape () doesn't match the broadcast shape (10,)